In [2]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [3]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [4]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [ ]:
model_tag = 'llama3'  
dataset_tag = 'yahoo'
prompt = 'cot_random_samples_cohere'
temp = 0.3
number_of_experimet = 1
selected_samples =  f'{model_tag}_random42_60'

In [6]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [7]:
dataset_for_annotation.columns

Index(['label', 'text'], dtype='object')

In [8]:
prompt_txt

'Task: Read the following text and determine the most appropriate category based on its content and the examples below.\n\nCategories:\n- World\n- Sports\n- Business\n- Sci/Tech\n\nExamples:\n{examples}\n\nText for you: "{text}"\n\nThink step by step. Then, give your final answer clearly marked like this: [category] '

In [9]:
examples_for_prompt

,label,text
0,Sci/Tech,FireFox 1.0 Roundup \\So FireFox 1.0 is releas...
1,Business,UPDATE 1-Potlatch to sell \$457.5 mln in asset...
2,Sci/Tech,UM scientist helping explore Saturn moon MISSO...
3,Sci/Tech,Asteroid Toutatis Makes Closest Pass in 651 Ye...
4,Business,Russian beer advertisements canned Putin signs...
...,...,...
75,Sci/Tech,"More on the INDUCE Act \\""Copyright officials ..."
76,World,Bush Twins Host Pre-Convention Party (AP) AP -...
77,World,8 Terror Suspects Appear in British Court LOND...
78,Business,"Wall Street Faces 2 Elections, Not Just 1 (Reu..."


In [10]:
text_col = "text"
label_col = "label"
output_col = "output"

In [11]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation[:100],
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [12]:
df = annotator.get_results()

/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: To categorize the provided text, we need to analyze its content. The text discusses BHP Billiton's plans to significantly increase its coking coal production in response to rising demand, particularly from China. This involves financial figures and strategic business decisions related to mining and production.

Given the focus on corporate plans, production increases, and market demand, this text clearly aligns with the category of Business.

Final answer: [Business]
Nr: 1 Predicted label: The text discusses a significant increase in energy bills due to the rising wholesale gas prices, which also affects electricity costs. This topic is related to the energy market and its impact on consumers, which falls under the category of Business.

Final answer: [Business]
Nr: 2 Predicted label: To categorize the provided text, we need to analyze its content. The text discusses a leak in a tunnel related to Boston's Big Dig highway project, which is a significant infrastruc

In [13]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/cot_random_samples_temp0.3/gpt4omini/ag_news_gpt4omini_cot_random_samples_gpt4omini_random42_80_temp0.3_exp1.csv


In [14]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label
0,Coke is it: BHP coal to double BHP Billiton pl...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
1,BIG HIKE IN ENERGY BILLS The company says the ...,The text discusses a significant increase in e...,"{'content': [{'token': 'The', 'bytes': [84, 10...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
2,Officials Blame Contractors in Tunnel Leak The...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business
4,Medical Examiner Finds No Injuries on Thanou-S...,To determine the appropriate category for the ...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports
...,...,...,...,...,...
95,"McNair, Brown Questionable Injuries again kept...","To categorize the provided text, let's analyze...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports
96,Briefly: Piracy crackdown yields \$2.2 million...,"To categorize the provided text, let's analyze...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sci/Tech
97,Sharon #39;s bold move in Gaza Tuesday #39;s v...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World
98,Darfur Mediators Demand Respect of Cease-fire ...,"To categorize the provided text, we first anal...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World


In [15]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_first_bracketed(text):
        match = re.search(r'\[([^\[\]]+?)\]', str(text))
        return match.group(1) if match else None
    
    df[write_col] = df[read_col].apply(extract_first_bracketed)
    return df


In [16]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, output_col, 'extracted_label')
    df.to_csv(path, index=False)
    
    return df


In [17]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,Coke is it: BHP coal to double BHP Billiton pl...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
1,BIG HIKE IN ENERGY BILLS The company says the ...,The text discusses a significant increase in e...,"{'content': [{'token': 'The', 'bytes': [84, 10...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
2,Officials Blame Contractors in Tunnel Leak The...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Sci/Tech
3,Before the Bell: Krispy Kreme Down 11 Pct Shar...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Business,False,Business
4,Medical Examiner Finds No Injuries on Thanou-S...,To determine the appropriate category for the ...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports,False,Sports
...,...,...,...,...,...,...,...
95,"McNair, Brown Questionable Injuries again kept...","To categorize the provided text, let's analyze...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sports,False,Sports
96,Briefly: Piracy crackdown yields \$2.2 million...,"To categorize the provided text, let's analyze...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",Sci/Tech,False,Business
97,Sharon #39;s bold move in Gaza Tuesday #39;s v...,"To categorize the provided text, we need to an...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World,False,World
98,Darfur Mediators Demand Respect of Cease-fire ...,"To categorize the provided text, we first anal...","{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",World,False,World
